In [ ]:
#Перебор по количеству кластеров в разных алгоритмах кластеризации
from collections import defaultdict
silhouette_dict = defaultdict(list)
# цикл по числу кластеров: от 2 до 10
for n in range(2, 11):
    # инициализируем алгоритмы:
    KM = KMeans(n_clusters=n, random_state=42)
    gm = GaussianMixture(n_components=n, random_state=42)
    ac = AgglomerativeClustering(n_clusters=n)
    # создаём словарь, где ключи - названия алгоритмов, значения - сами алгоритмы:
    alg_dict = {'K-means': KM, 'EM-алгоритм': gm, 'Агломеративная кластеризация': ac}
    # цикл по словарю:
    for alg_name, algo in alg_dict.items():
        labels = algo.fit_predict(X) # получаем предсказания
        sil_score = silhouette_score(X, labels) # считаем коэффициент силуэта
        # добавляем в словарь в list, соответствующему рассматриваемому алгоритму, 
        # пару вида : (число кластеров, коэффициент силуэта)
        silhouette_dict[alg_name].append((n, sil_score)) 
# цикл по ключам словаря с коэффициентами силуэта для алгоритмов:
for alg_name in silhouette_dict.keys():
    # сохраняем число кластеров и коэф. силуэта для пары, 
    # в которой коэф. максимальный для данного алгоритма:
    n_clusters, sil_score = max(silhouette_dict[alg_name], key=lambda x: x[1])
    # выводим  название алгоритма и искомое число кластеров (и коэф. силуэта):
    print(f"{alg_name} : {n_clusters}, {sil_score}")

In [ ]:
#визуализация аглломеративного алгоритма
df = pd.read_csv("./food.csv", sep=' ')
X = df.drop('Name', axis=1)
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X)
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(X_transformed, method='average', metric='euclidean')
names = df.Name.values
dend = dendrogram(Z, color_threshold=0, labels=names, 
                  orientation='left')
from scipy.cluster.hierarchy import fcluster
t = 2.3
labels = fcluster(Z, t, criterion='distance')
print(len(np.unique(labels)))

# Стекинг

In [ ]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    разбивает выборку на фолды и обчает указанный алгоритм на cv-1 фолде
    
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [ ]:
def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

In [ ]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_tests_array = []
    splits = 0
    for train_fold_index, predict_fold_index in cv.split(X_train):
        n_classes = len(np.unique(y_test))
        X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
        splits += 1
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_tests_array.append(folded_clf.predict_proba(X_test))
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    X_meta_test = sum(X_meta_tests_array) / splits
    return X_meta_train, X_meta_test

# Статистика

In [ ]:
#Тест стьюдента
#
data_new=data.copy()
def get_stat_dif(column):
    cols = data[data['Sample']==1].loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(data[data['Sample']==1].loc[data_new.loc[:, column] == comb[0], 'Rating'],
                     data[data['Sample']==1].loc[data_new.loc[:, column] == comb[1], 'Rating']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
        else:
            print('Не найдены статистически значимые различия для колонки', column)
            break

            
for col in data_new.columns.tolist():
     get_stat_dif(col)

In [ ]:
def dataset_info(df):
    print('Размерность датасета: ', df.shape)
    display(df.head(5))
    df.info()

In [1]:
def describe_nan_values(d_df):
    total = d_df.isnull().sum().sort_values(ascending=False)
    percent = (d_df.isnull().sum()/d_df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    display(missing_data[missing_data['Total']>0])
    
    return


# Линейные параметры

In [ ]:
def describe_without_plots_num_collumns(d_df,columns):
    #numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_df = d_df[columns]
    list_of_names = list(num_df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    temp_dict['тип'] = num_df.dtypes
    temp_dict['# пропусков(NaN)'] = num_df.isnull().sum().values 
    temp_dict['# уникальных'] = num_df.nunique().values
    temp_dict['минимум'] = num_df.describe(include='all').loc['min']
    temp_dict['среднее'] = num_df.describe(include='all').loc['mean']
    temp_dict['медиана'] = num_df.describe(include='all').loc['50%']
    temp_dict['макс'] = num_df.describe(include='all').loc['max']
    temp_dict['std'] = pd.DataFrame(num_df.std().values,columns=['std']).loc[:,'std']
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)
    
    return

In [ ]:
def describe_by_plots_num_collumns(d_df,columns,font_size=12):
    fig, axes = plt.subplots(len(columns), 4, figsize=(8, 5*len(columns)))
    for i in range(0,len(columns)):
        axes[i,0].hist(d_df[columns[i]],label=str(columns[i]))
        axes[i,0].set_title(str(columns[i]),fontsize=font_size)
        axes[i,1].boxplot(d_df[columns[i]])
        axes[i,1].set_title(str(columns[i]),fontsize=font_size)
        axes[i,2].hist(np.log(d_df[columns[i]]+1),label=str(columns[i]))
        axes[i,2].set_title('log_'+str(columns[i]),fontsize=font_size)
        axes[i,3].boxplot(np.log(d_df[columns[i]]+1))
        axes[i,3].set_title('log_'+str(columns[i]),fontsize=font_size)
    return

In [ ]:
import math

def borders_of_outliers(d_df,columns, log = False):
    num_df = d_df[columns]
    list_of_names = list(num_df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    left_board=[]
    right_board=[]
    left_outliers=[]
    right_outliers=[]
    path_left_outliers=[]
    path_right_outliers=[]
    if log:
        log_left_board=[]
        log_right_board=[]
        log_left_outliers=[]
        log_right_outliers=[]
        log_path_left_outliers=[]
        log_path_right_outliers=[]
    for i in columns:
        IQR = num_df[i].quantile(0.75) - num_df[i].quantile(0.25)
        perc25 = num_df[i].quantile(0.25)
        perc75 = num_df[i].quantile(0.75)
        left_border = num_df[i].quantile(0.25) - 1.5*IQR
        right_border = perc75 + 1.5*IQR
        left_board.append(left_border)
        right_board.append(right_border)
        left_outliers.append((num_df[i]<left_border).sum())
        right_outliers.append((num_df[i]>right_border).sum())
        path_left_outliers.append((num_df[i]<left_border).sum()/len(num_df[i]))
        path_right_outliers.append((num_df[i]>right_border).sum()/len(num_df[i]))
        if log:
            num_df[i]=num_df[i].apply(lambda x: math.log(x+1))
            IQR = num_df[i].quantile(0.75) - num_df[i].quantile(0.25)
            perc25 = num_df[i].quantile(0.25)
            perc75 = num_df[i].quantile(0.75)
            left_border = num_df[i].quantile(0.25) - 1.5*IQR
            right_border = perc75 + 1.5*IQR
            log_left_board.append(left_border)
            log_right_board.append(right_border)
            log_left_outliers.append((num_df[i]<left_border).sum())
            log_right_outliers.append((num_df[i]>right_border).sum())
            log_path_left_outliers.append((num_df[i]<left_border).sum()/len(num_df[i]))
            log_path_right_outliers.append((num_df[i]>right_border).sum()/len(num_df[i]))
        
    temp_dict['левая граница'] = left_board
    temp_dict['правая граница'] = right_board
    temp_dict['выбросов слева'] = left_outliers
    temp_dict['выбросов справа'] = right_outliers
    temp_dict['доля выбросов слева'] = path_left_outliers
    temp_dict['доля выбросов справа'] = path_right_outliers
    
    if log:
        temp_dict['левая граница с логарифмом'] = log_left_board
        temp_dict['правая граница с логарифмом'] = log_right_board
        temp_dict['выбросов слева с логарифмом'] = log_left_outliers
        temp_dict['выбросов справа с логарифмом'] = log_right_outliers
        temp_dict['доля выбросов слева с логарифмом'] = log_path_left_outliers
        temp_dict['доля выбросов справа с логарифмом'] = log_path_right_outliers
    
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)
    
    return

In [2]:
def num_describer(df,param,bins=20):
    nulls=round(df[param].isnull().sum()/len(df[param]),4)
    low = df[param].min()
    median = df[param].median()
    mean = df[param].mean()
    hight = df[param].max()
    IQR = df[param].quantile(
        0.75) - df[param].quantile(0.25)
    perc25 = df[param].quantile(0.25)
    perc75 = df[param].quantile(0.75)
    #blowout=len(df[(df[param]<perc25 - 1.5*IQR)|(df[param]>perc75 + 1.5*IQR)])
    print('доля пропусков : {} \n\r'.format(nulls),
          'min : {} \n\r'.format(low),
          '25-й перцентиль: {} \n\r'.format(perc25),
          'медиана: {} \n\r'.format(median),
          'среднее: {} \n\r'.format(mean),
          'max : {} \n\r'.format(hight),
          '75-й перцентиль: {} \n\r'.format(perc75),
          "IQR: {} \n\r".format(IQR),
          "Границы выбросов: [{f}, {l}] \n\r".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR),
         )#'Количество выбросов: \n\r'.format(blowout))
    fig, (ax1, ax2) = plt.subplots(
    nrows=1, ncols=2,
    figsize=(8, 4))
    sns.distplot(df[param],ax=ax1)
    #ax1.hist(df[param],label = param,bins=bins)
    ax1.set_title('Распределение значений')
    ax1.legend()
    sns.distplot(np.log(df[param]+1),ax=ax2)
    #ax2.hist(np.log(df[param]+1),label = param,bins=bins)
    ax2.set_title('Логорифмированное Распределение значений')
    ax2.legend()
    #sns.distplot(df[param])
    plt.show()

# Категориальные и биномиальные

In [ ]:
def describe_without_plots_cat_bin_collumns(d_df,columns):
    cat_bin_df = d_df[columns]
    list_of_names = list(cat_bin_df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    temp_dict['тип'] = cat_bin_df.dtypes
    temp_dict['# пропусков(NaN)'] = cat_bin_df.isnull().sum().values 
    temp_dict['# уникальных'] = cat_bin_df.nunique().values
    temp_dict['# мода'] = cat_bin_df.mode()
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)
    
    return

In [ ]:
def obj_param_describe(df,param):
    print('доля пропусков: ',df[param].isnull().sum()/len(df[param]))
    print('Значения параметра и их количество')
    df[param].value_counts().plot.bar()
    print(df[param].value_counts())
    fig, ax = plt.subplots(figsize=(6, 6))
    sns.boxplot(x=param, y='default', data=df)

# Regression

In [ ]:
#Рисуем результат работы регрессионной модели
def Regression_plot(X,y,X_test,Y_test):
    #X- все примеры
    #У- все метки
    #X_test - тестовая/обучающая выборка нашеймодели
    #Y_test - результат работы модели
    plt.figure()
    #Отрисовываем наши точки
    plt.scatter(X, y, s=20, edgecolor="black",
                c="darkorange", label="data")
    #Отрисовываем кривую модели
    plt.plot(X_test, Y_test, color="yellowgreen", label="Model", linewidth=2)
    plt.xlabel("data")
    plt.ylabel("target")
    plt.title("Decision Tree Regression")
    plt.legend()
    plt.show()

Regression_plot(X,y,X_test,y_2)

# Decision Tree 

In [ ]:
#Визуализация дерева без текста
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 20))
tree.plot_tree(model, max_depth=5, fontsize=10)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import random
random.seed(42)

#Показывает качество работы модели в зависимости от параметра

def get_dependency_and_plot(parameters, param_title=None):
    # parameters: tuple, где первый элемент - название параметра RandomForestClassifier, (например, n_estimators),
    # а второй - диапазон для заданного параметра.
    #
    # param_title: Продолжение фразы 'Зависимость accuracy от ...' из заголовка графика.
    # Опциональный параметр. Если None, то заголовок не оторбражается
    param_name, param_range = parameters
    accuracy_list = []
    # цикл по диапазону параметра:
    for value in param_range:
        # словарь параметров для задания параметров модели через метод set_params()
        param_dict = {param_name: value}
        model = RandomForestClassifier()
        model.set_params(**param_dict)
        # считаем accuracy на кросс-валидации и берём среднее:
        score = cross_val_score(model, X, Y, cv=10, n_jobs=-1).mean()
        accuracy_list.append((value, score))
    # отрисовываем график:
    xx = [z[0] for z in accuracy_list]
    yy = [z[1] for z in accuracy_list]
    plt.plot(xx, yy, '.-')
    if param_title != None:
        plt.title('Зависимость accuracy от ' + param_title)
    plt.xlabel(param_name)
    plt.ylabel("Mean accuracy")


n_estimators_range = np.arange(1, 200, 10, dtype=int)
max_features_range = np.arange(5, X.shape[1], 5, dtype=int)
max_depth_range = np.arange(3, 30, 2, dtype=int)
# Также определим пары tuple для работы с определённой выше функцией:
est_tuple = ('n_estimators', n_estimators_range)
max_features_tuple = ('max_features', max_features_range)
max_depth_tuple = ('max_depth', max_depth_range)